# Capstone Project

## Introduction

For my capstone project I want to try to use machine learning to aggregate information from different financial analytical approaches 

Tesla's disruption to the automotive industry began back in 2008 with its exclusive highend Roadster () up until the recent reveal of the mainstream Model Y ().

As the pioneer and leader in EV sales worldwide (), Tesla perhaps has gained more attention in its financial sustainability (); as of this writing, Tesla has yet produced a profittable quarter(). This ***uncertainity of unprofitablity*** has resonated throughout the EV market and automotive market in large, which in turn made Tesla to become one of the most important benchmark indicator of the EV story (). 

Indeed, analysts has been extermely diverged in the valuation of this market leader (). This report will attempt to use machine learning to re-evaluate Tesla using three distinct financial analytical approaches: 
    1) Intrinsic Value (Fundamental Analysis); 2) Discounted Cash Flow; 3) Comparable analysis; and 4) Time series Analysis.
    
In this report I will attempt to predict the Tesla's stock price using different financial and statistical models. In the end I will discuss the performance of each model and some future recommendations. 

### **Investment Horizon of Models** 

1) **Intrinsic Value** Long Term (Quarters/Years)

2) **Discounted Cash Flow** Mid Term (Quarters)

3) **Comparaable Analysis** Mid Term (Days/Months)

4) **Time Series** Short (Days)

    - Time Autoregressive Integrated Moving Average (ARIMA)
     https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3168423

Model Evalution
AUROC


# Libraries

In [1]:
# Import Library 
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from pprint import pprint
import numpy
from numpy import array
import pandas as pd 
import csv
import pickle
from IPython.display import display


# Define Text Normalization Function

In [2]:
 def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

# Grab the Document Content

In [3]:
# Name: CIK: HTML
company = {}
company['auto'] = {}
Accession_Number_URL = {}
file_url = {
    '0001318605' : {}}
master_filings_dict = {}

master_document_dict = {}
#print(company['name'])
file_code = {}
file_text = {}

auto = {'Tesla' : '0001318605'
       }
        
company['auto'] = auto

CIK = {
    '0001318605' : '{}'    }


key = list(CIK.copy().keys())


print('Company Dictionary', company)
print('-'*40)

print( 'Auto: {}'.format( company['auto'] ) )

print('-'*80)

# List of CIK
print( 'CIK-Keys: {}'.format(key) )
print('-'*80)


Company Dictionary {'auto': {'Tesla': '0001318605'}}
----------------------------------------
Auto: {'Tesla': '0001318605'}
--------------------------------------------------------------------------------
CIK-Keys: ['0001318605']
--------------------------------------------------------------------------------


In [4]:
# URL Directory For CIK    
dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'
dir_url_list = [dir_url.format(x) for x in key]

#print('Directory URL: {}'.format(dir_url_list))
doc_url_list = [] 

# FOR-loop yielding Accession Numbers from CIK/URL Directory.
for CIK_num in key:

    doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'
    doc_url_new = doc_url.format(CIKx = CIK_num, xx='{xx}', yy ='{yy}')
    doc_url_list.append(doc_url_new)
print(doc_url_list)

['https://www.sec.gov/Archives/edgar/data/0001318605/{xx}/{yy}.txt']


## Create Accession Number List: file_url

In [5]:
for url in dir_url_list:
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    text = soup.get_text(strip=True)

    cleaned_text = re.findall('Acc-no: \d+-\d+-\d+' , text)

    accession_number = [n.replace('Acc-no: ', '') for n in cleaned_text]
    accessionnumber = [num.replace('-', '') for num in accession_number]

    accession_numbers = zip(accessionnumber, accession_number)
    
    cikk = [cikk_.replace('CIK=', '') for cikk_ in re.findall('CIK=\d+', url)][0]
    CIKK = {cikk : accessionnumber}
    CIK.update(CIKK)
    
    for (a,b) in accession_numbers: 
            master_filings_dict[b] = {}
            master_filings_dict[b]['sec_header_content'] = {}
            master_filings_dict[b]['filing_documents'] = None
            
            doc_url_single = doc_url_list[key.index(cikk)].format(xx = a, yy = b)
            
            #Checking extracted URL CIK number matches with in-house Dictionary CIK number.
            rotat = key.index(cikk)
            #print('rotat: ', rotat)
            
            doc_url_str = doc_url_list[ rotat ]
            #print(doc_url_str, type(doc_url_str))
            cik = [data.replace('/data/','') for data in re.findall('/data/\d+', doc_url_str )][0].strip()
            #print('cik', cik, ':', 'cikk', cikk)

            file_url_list = []
            Accession_Number_URL = {}


            if (cikk == cik) == True:
            
                file_url_list.append( doc_url_single )
                Accession_Number_URL.update({ b : file_url_list})
                
                file_url[cikk].update(Accession_Number_URL)
                
                
                #print('cik{}: {}'.format((rotat +1), cik))  
    
    
            else:
                print('An error occured while checking!', 'cik: ', cik, 'cikk: ', cikk, 'Accession_Number: ', a)
                print('~~~~~'*20)
                
            del file_url_list
            del b 
            del Accession_Number_URL
                #company['auto']['CIK'] = Accession_Number_URL

In [6]:
A_N = []
for AN, url in file_url['0001318605'].items():
    
    if AN == '0001193125-14-403635':
        break
        
    else:
        A_N.append(AN)
print(A_N)
print(url)

['0001564590-20-004475', '0001564590-19-038256', '0001564590-19-026445', '0001564590-19-013462', '0001564590-19-003165', '0001564590-18-026353', '0001564590-18-019254', '0001564590-18-011086', '0001564590-18-002956', '0001564590-17-021343', '0001564590-17-015705', '0001564590-17-009968', '0001564590-17-003118', '0001564590-16-026820', '0001564590-16-023024', '0001564590-16-018886', '0001564590-16-013195', '0001564590-15-009741', '0001564590-15-006666', '0001564590-15-003789', '0001564590-15-001031']
['https://www.sec.gov/Archives/edgar/data/0001318605/000119312514403635/0001193125-14-403635.txt']


## For each url, scrape and save in master_filings_dict[AN]['filing_documents']

In [7]:
for acc_num, url in file_url['0001318605'].items():
            if acc_num == '0001193125-14-403635':
                break
            
            # grab the response
            response = requests.get(url[0])

            # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
            soup = BeautifulSoup(response.content, 'lxml')
            filing_document = soup.find('document')
            
            # Document type ->> document_id
            document_id = filing_document.type.find(text=True, recursive=False).strip()
            # Document sequence ->> document_sequence
            #document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
            # Document filename ->> document_filename
            document_filename = filing_document.filename.find(text=True, recursive=False).strip()
            # Document description ->> document_description
            document_description = filing_document.description.find(text=True, recursive=False).strip()
            
            # initalize our document dictionary
            master_document_dict[document_id] = {}

            # add the different parts, we parsed up above.
            #master_document_dict[document_id]['document_sequence'] = document_sequence
            master_document_dict[document_id]['document_filename'] = document_filename
            master_document_dict[document_id]['document_description'] = document_description

            # store the document itself, this portion extracts the HTML code. We will have to reparse it later.
            master_document_dict[document_id]['document_code'] = filing_document.extract()

            # grab the text portion of the document, this will be used to split the document into pages.
            filing_doc_text = filing_document.find('text').extract()


            # find all the thematic breaks, these help define page numbers and page breaks.
            all_thematic_breaks = filing_doc_text.find_all('hr')
            

            
            all_page_numbers = [thematic_break.previous_sibling.previous_sibling.get_text(strip=True) for thematic_break in all_thematic_breaks]
            
                
       # '''
       # -------------------------------
         # THE OPTIONAL CODE HAS STARTED
        #------------------------------- 
    #'''
        
            # determine the number of pages, will be used for the upcoming if conditions.
            length_of_page_numbers = len(all_page_numbers)
    
            # as long as there are numbers to change then proceed.
            if length_of_page_numbers > 0:
        
                # grab the last number
                previous_number = all_page_numbers[-1]
        
                # initalize a new list
                all_page_numbers_cleaned = []
        
                # loop through the old list in reverse order.
                for number in reversed(all_page_numbers):
            
                    # if it's blank proceed to cleaning.
                    if number == '':
                
                        # the tricky part, there are three scenarios.

                        # the previous one we looped was 0 or 1.
                        if previous_number == '1' or previous_number == '0':
                    
                            # in this case, it means this is a "new section", so restart at 0.
                            all_page_numbers_cleaned.append(str(0))
                    
                            # reset the page number and the previous number.
                            length_of_page_numbers = length_of_page_numbers - 1
                            previous_number = '0'
                
                        # the previous one we looped it wasn't either of those.
                        else:
                    
                            # if it was blank, take the current length, subtract 1, and add it to the list.
                            all_page_numbers_cleaned.append(str(length_of_page_numbers - 1))
                    
                            # reset the page number and the previous number.
                            length_of_page_numbers = length_of_page_numbers - 1
                            previous_number = number

                    else:
                
                        # add the number to the list.
                        all_page_numbers_cleaned.append(number)
                
                        # reset the page number and the previous number.
                        length_of_page_numbers = length_of_page_numbers - 1
                        previous_number = number
            else:
        
                # make sure that it has a page number even if there are none, just have it equal 0
                all_page_numbers_cleaned = ['0']
    
            # have the page numbers be the cleaned ones, in reversed order.
            all_page_numbers = list(reversed(all_page_numbers_cleaned))
    
            # store the page_numbers
            master_document_dict[document_id]['page_numbers'] = all_page_numbers
    
    
    #''''
       # -------------------------------
        #  THE OPTIONAL CODE HAS ENDED
      #  ------------------------------- 

    #''''
            # convert all thematic breaks to a string so it can be used for parsing
            all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]
            
            # prep the document text for splitting, this means converting it to a string.
            filing_doc_string = str(filing_doc_text)
    ####################################################################
            # handle the case where there are thematic breaks.
            if len(all_thematic_breaks) > 0:

                # define the regex delimiter pattern, this would just be all of our thematic breaks.
                regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

                # split the document along each thematic break.
                split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

                # store the document itself
                master_document_dict[document_id]['pages_code'] = split_filing_string

            # handle the case where there are no thematic breaks.
            elif len(all_thematic_breaks) == 0:

                # handles so it will display correctly.
                split_filing_string = all_thematic_breaks

                # store the document as is, since there are no thematic breaks. In other words, no splitting.
                master_document_dict[document_id]['pages_code'] = [filing_doc_string]

            # display some information to the user.
            print('-'*80)
            print('The document {} was parsed.'.format(document_id))
            print('There was {} page(s) found.'.format(len(all_page_numbers)))
            print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))
    
            # store in local disc
            #urllib.request.urlretrieve("http://www.example.com/test.html", "test.txt")

            # store the documents in the master_filing_dictionary.
            master_filings_dict[acc_num]['filing_documents'] = master_document_dict

            print('-'*80)
            print('All the documents for filing {} were parsed and stored.'.format(acc_num))


--------------------------------------------------------------------------------
The document 10-K was parsed.
There was 158 page(s) found.
There was 158 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001564590-20-004475 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 72 page(s) found.
There was 72 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001564590-19-038256 were parsed and stored.
--------------------------------------------------------------------------------
The document 10-Q was parsed.
There was 70 page(s) found.
There was 70 thematic breaks(s) found.
--------------------------------------------------------------------------------
All the documents for filing 0001564590-19-026445 were parsed and stored.

# Normalising Text


In [7]:
for acc_num in A_N:
    # first grab all the documents
    filing_documents = master_filings_dict[acc_num]['filing_documents']
    print(acc_num)

    # loop through each document
    for document_id in filing_documents:

        # display some info to give status updates.
        print('-'*80)
        print('Pulling document {} for text normilzation.'.format(document_id))

        # grab all the pages for that document, stored in 'pages_code' key
        document_pages = filing_documents[document_id]['pages_code']
        print(document_pages)
        break
        for index, page in enumerate(document_pages):
            page_soup = BeautifulSoup(page,'html5')
            print('{} in {} has been parsed to HTML5'.format(document_id,acc_num))
            print(index)

0001564590-20-004475


TypeError: 'NoneType' object is not iterable

In [8]:
for acc_num in A_N:
    # first grab all the documents
    filing_documents = master_filings_dict[acc_num]['filing_documents']
    print(acc_num)

    # loop through each document
    for document_id in filing_documents:

        # display some info to give status updates.
        print('Pulling document {} for text normilzation.'.format(document_id))

        # grab all the pages for that document, stored in 'pages_code' key
        document_pages = filing_documents[document_id]['pages_code']
    
        # page length
        pages_length = len(filing_documents[document_id]['pages_code'])

        # initalize a dictionary that'll house our repaired html code for each page.
        repaired_pages = {}

        # initalize a dictionary that'll house all the normalized text.
        normalized_text = {}

        # loop through each page in that document.
        for index, page in enumerate(document_pages):

            # pass it through the parser. NOTE I AM USING THE HTML5 PARSER. YOU MUST USE THIS TO FIX BROKEN TAGS.
            page_soup = BeautifulSoup(page,'html5')

            # grab all the text, notice I go to the BODY tag to do this
            page_text = page_soup.html.body.get_text(' ',strip = True)

            # normalize the text, remove messy characters. Additionally, restore missing window characters.
            page_text_norm = restore_windows_1252_characters(unicodedata.normalize('NFKD', page_text)) 

            # Additional cleaning steps, removing double spaces, and new line breaks.
            page_text_norm = page_text_norm.replace('  ', ' ').replace('\n',' ')


            '''
                NOTES FROM UP ABOVE:
                --------------------

                Remember up above, where I had some optional code. Well, this is where we add page numbers. If you notice
                I simply take the index add 1 to it and with that we now have a page number. Now, this doesn't technically
                follow the sections in each document but I don't think most people will care. Also we will see that we can
                infer the sections from other parts.

            '''

            # define the page number.
            page_number = index + 1

            # add the normalized text to the list.
            normalized_text[page_number] = page_text_norm

            # add the repaired html to the list. Also now we have a page number as the key.
            repaired_pages[page_number] = page_soup

            # display a status to the user
            print('Page {} of {} from document {} has had their text normalized.'.format(index + 1, 
                                                                                         pages_length, 
                                                                                         document_id))
            del page_soup
        # add the normalized text back to the document dictionary
        filing_documents[document_id]['pages_normalized_text'] = normalized_text

        # add the repaired html code back to the document dictionary
        filing_documents[document_id]['repaired_pages_code'] = repaired_pages

        # define the generated page numbers
        gen_page_numbers = list(repaired_pages.keys())

        # add the page numbers we have.
        filing_documents[document_id]['pages_numbers_generated'] = gen_page_numbers    

        # display a status to the user.
        print('All the pages from document {} have been normalized.'.format(document_id))
        print('-'*80)

0001564590-20-004475
Pulling document 10-K for text normilzation.
Page 1 of 98 from document 10-K has had their text normalized.
Page 2 of 98 from document 10-K has had their text normalized.
Page 3 of 98 from document 10-K has had their text normalized.
Page 4 of 98 from document 10-K has had their text normalized.
Page 5 of 98 from document 10-K has had their text normalized.
Page 6 of 98 from document 10-K has had their text normalized.
Page 7 of 98 from document 10-K has had their text normalized.
Page 8 of 98 from document 10-K has had their text normalized.
Page 9 of 98 from document 10-K has had their text normalized.
Page 10 of 98 from document 10-K has had their text normalized.
Page 11 of 98 from document 10-K has had their text normalized.
Page 12 of 98 from document 10-K has had their text normalized.
Page 13 of 98 from document 10-K has had their text normalized.
Page 14 of 98 from document 10-K has had their text normalized.
Page 15 of 98 from document 10-K has had their 

Page 29 of 53 from document 10-Q has had their text normalized.
Page 30 of 53 from document 10-Q has had their text normalized.
Page 31 of 53 from document 10-Q has had their text normalized.
Page 32 of 53 from document 10-Q has had their text normalized.
Page 33 of 53 from document 10-Q has had their text normalized.
Page 34 of 53 from document 10-Q has had their text normalized.
Page 35 of 53 from document 10-Q has had their text normalized.
Page 36 of 53 from document 10-Q has had their text normalized.
Page 37 of 53 from document 10-Q has had their text normalized.
Page 38 of 53 from document 10-Q has had their text normalized.
Page 39 of 53 from document 10-Q has had their text normalized.
Page 40 of 53 from document 10-Q has had their text normalized.
Page 41 of 53 from document 10-Q has had their text normalized.
Page 42 of 53 from document 10-Q has had their text normalized.
Page 43 of 53 from document 10-Q has had their text normalized.
Page 44 of 53 from document 10-Q has had

Page 4 of 53 from document 10-Q has had their text normalized.
Page 5 of 53 from document 10-Q has had their text normalized.
Page 6 of 53 from document 10-Q has had their text normalized.
Page 7 of 53 from document 10-Q has had their text normalized.
Page 8 of 53 from document 10-Q has had their text normalized.
Page 9 of 53 from document 10-Q has had their text normalized.
Page 10 of 53 from document 10-Q has had their text normalized.
Page 11 of 53 from document 10-Q has had their text normalized.
Page 12 of 53 from document 10-Q has had their text normalized.
Page 13 of 53 from document 10-Q has had their text normalized.
Page 14 of 53 from document 10-Q has had their text normalized.
Page 15 of 53 from document 10-Q has had their text normalized.
Page 16 of 53 from document 10-Q has had their text normalized.
Page 17 of 53 from document 10-Q has had their text normalized.
Page 18 of 53 from document 10-Q has had their text normalized.
Page 19 of 53 from document 10-Q has had their

Page 82 of 98 from document 10-K has had their text normalized.
Page 83 of 98 from document 10-K has had their text normalized.
Page 84 of 98 from document 10-K has had their text normalized.
Page 85 of 98 from document 10-K has had their text normalized.
Page 86 of 98 from document 10-K has had their text normalized.
Page 87 of 98 from document 10-K has had their text normalized.
Page 88 of 98 from document 10-K has had their text normalized.
Page 89 of 98 from document 10-K has had their text normalized.
Page 90 of 98 from document 10-K has had their text normalized.
Page 91 of 98 from document 10-K has had their text normalized.
Page 92 of 98 from document 10-K has had their text normalized.
Page 93 of 98 from document 10-K has had their text normalized.
Page 94 of 98 from document 10-K has had their text normalized.
Page 95 of 98 from document 10-K has had their text normalized.
Page 96 of 98 from document 10-K has had their text normalized.
Page 97 of 98 from document 10-K has had

Page 65 of 98 from document 10-K has had their text normalized.
Page 66 of 98 from document 10-K has had their text normalized.
Page 67 of 98 from document 10-K has had their text normalized.
Page 68 of 98 from document 10-K has had their text normalized.
Page 69 of 98 from document 10-K has had their text normalized.
Page 70 of 98 from document 10-K has had their text normalized.
Page 71 of 98 from document 10-K has had their text normalized.
Page 72 of 98 from document 10-K has had their text normalized.
Page 73 of 98 from document 10-K has had their text normalized.
Page 74 of 98 from document 10-K has had their text normalized.
Page 75 of 98 from document 10-K has had their text normalized.
Page 76 of 98 from document 10-K has had their text normalized.
Page 77 of 98 from document 10-K has had their text normalized.
Page 78 of 98 from document 10-K has had their text normalized.
Page 79 of 98 from document 10-K has had their text normalized.
Page 80 of 98 from document 10-K has had

...

Page 56 of 98 from document 10-K has had their text normalized.
Page 57 of 98 from document 10-K has had their text normalized.
Page 58 of 98 from document 10-K has had their text normalized.
Page 59 of 98 from document 10-K has had their text normalized.
Page 60 of 98 from document 10-K has had their text normalized.
Page 61 of 98 from document 10-K has had their text normalized.
Page 62 of 98 from document 10-K has had their text normalized.
Page 63 of 98 from document 10-K has had their text normalized.
Page 64 of 98 from document 10-K has had their text normalized.
Page 65 of 98 from document 10-K has had their text normalized.
Page 66 of 98 from document 10-K has had their text normalized.
Page 67 of 98 from document 10-K has had their text normalized.
Page 68 of 98 from document 10-K has had their text normalized.
Page 69 of 98 from document 10-K has had their text normalized.
Page 70 of 98 from document 10-K has had their text normalized.
Page 71 of 98 from document 10-K has had

Page 42 of 98 from document 10-K has had their text normalized.
Page 43 of 98 from document 10-K has had their text normalized.
Page 44 of 98 from document 10-K has had their text normalized.
Page 45 of 98 from document 10-K has had their text normalized.
Page 46 of 98 from document 10-K has had their text normalized.
Page 47 of 98 from document 10-K has had their text normalized.
Page 48 of 98 from document 10-K has had their text normalized.
Page 49 of 98 from document 10-K has had their text normalized.
Page 50 of 98 from document 10-K has had their text normalized.
Page 51 of 98 from document 10-K has had their text normalized.
Page 52 of 98 from document 10-K has had their text normalized.
Page 53 of 98 from document 10-K has had their text normalized.
Page 54 of 98 from document 10-K has had their text normalized.
Page 55 of 98 from document 10-K has had their text normalized.
Page 56 of 98 from document 10-K has had their text normalized.
Page 57 of 98 from document 10-K has had

Page 17 of 98 from document 10-K has had their text normalized.
Page 18 of 98 from document 10-K has had their text normalized.
Page 19 of 98 from document 10-K has had their text normalized.
Page 20 of 98 from document 10-K has had their text normalized.
Page 21 of 98 from document 10-K has had their text normalized.
Page 22 of 98 from document 10-K has had their text normalized.
Page 23 of 98 from document 10-K has had their text normalized.
Page 24 of 98 from document 10-K has had their text normalized.
Page 25 of 98 from document 10-K has had their text normalized.
Page 26 of 98 from document 10-K has had their text normalized.
Page 27 of 98 from document 10-K has had their text normalized.
Page 28 of 98 from document 10-K has had their text normalized.
Page 29 of 98 from document 10-K has had their text normalized.
Page 30 of 98 from document 10-K has had their text normalized.
Page 31 of 98 from document 10-K has had their text normalized.
Page 32 of 98 from document 10-K has had

Page 4 of 98 from document 10-K has had their text normalized.
Page 5 of 98 from document 10-K has had their text normalized.
Page 6 of 98 from document 10-K has had their text normalized.
Page 7 of 98 from document 10-K has had their text normalized.
Page 8 of 98 from document 10-K has had their text normalized.
Page 9 of 98 from document 10-K has had their text normalized.
Page 10 of 98 from document 10-K has had their text normalized.
Page 11 of 98 from document 10-K has had their text normalized.
Page 12 of 98 from document 10-K has had their text normalized.
Page 13 of 98 from document 10-K has had their text normalized.
Page 14 of 98 from document 10-K has had their text normalized.
Page 15 of 98 from document 10-K has had their text normalized.
Page 16 of 98 from document 10-K has had their text normalized.
Page 17 of 98 from document 10-K has had their text normalized.
Page 18 of 98 from document 10-K has had their text normalized.
Page 19 of 98 from document 10-K has had their

Page 14 of 98 from document 10-K has had their text normalized.
Page 15 of 98 from document 10-K has had their text normalized.
Page 16 of 98 from document 10-K has had their text normalized.
Page 17 of 98 from document 10-K has had their text normalized.
Page 18 of 98 from document 10-K has had their text normalized.
Page 19 of 98 from document 10-K has had their text normalized.
Page 20 of 98 from document 10-K has had their text normalized.
Page 21 of 98 from document 10-K has had their text normalized.
Page 22 of 98 from document 10-K has had their text normalized.
Page 23 of 98 from document 10-K has had their text normalized.
Page 24 of 98 from document 10-K has had their text normalized.
Page 25 of 98 from document 10-K has had their text normalized.
Page 26 of 98 from document 10-K has had their text normalized.
Page 27 of 98 from document 10-K has had their text normalized.
Page 28 of 98 from document 10-K has had their text normalized.
Page 29 of 98 from document 10-K has had

In [ ]:
pprint(master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['pages_normalized_text'])

# Normalising Text


In [ ]:
def normalising(filing_10K_or_10Q):

    # initalize a list that'll house all the normalized text.
    normalized_text = []
    
    # initalize a dictionary that'll house our repaired html code for each page.
    repaired_pages = {}
        
    # loop through each page in that document.
    for index, page in enumerate(filing_10K_or_10Q):

        # pass it through the parser. NOTE I AM USING THE HTML5 PARSER. YOU MUST USE THIS TO FIX BROKEN TAGS.
        page_soup = BeautifulSoup(page,'html5')

        # grab all the text, notice I go to the BODY tag to do this
        page_text = page_soup.html.body.get_text(' ',strip = True)

        # normalize the text, remove messy characters. Additionally, restore missing window characters.
        page_text_norm = restore_windows_1252_characters(unicodedata.normalize('NFKD', page_text)) 

        # Additional cleaning steps, removing double spaces, and new line breaks.
        page_text_norm = page_text_norm.replace('  ', ' ').replace('\n',' ')

        # define the page number.
        page_number = index + 1

        # add the repaired html to the list. Also now we have a page number as the key.
        repaired_pages[page_number] = page_soup

        # add the normalized text back to the document dictionary
        normalized_text.append(page_text_norm)

        del page_soup
        
    return normalized_text

# .savetext() file method 
    for acc_num in A_N:

        filing_documents = master_filings_dict[acc_num]['filing_documents']['10-Q']['pages_code']
        a = array(filing_documents)
        numpy.savetxt("{}.txt".format(acc_num), a, fmt='%s', encoding="utf-8")

        print(f"{acc_num} has been saved as numpy file in local disc.")


# pd.read_csv('0001564590-15-001031.csv')
pages_dict = []
for acc_num in A_N:

    f = open('{}.txt'.format(acc_num), 'r', encoding="utf-8")
    content = f.read()
    #print(content)
    for a in content:
        pages_dict.append(str(a))
    print(pages_dict)
    break

# Finding Tables 
### --> select the correct tables here!!!!

simplified version for one filing


>>> import pickle
>>> l = [1,2,3,4]
>>> with open("test.txt", "wb") as fp:   #Pickling
...   pickle.dump(l, fp)
... 
>>> with open("test.txt", "rb") as fp:   # Unpickling
...   b = pickle.load(fp)
... 
>>> b
[1, 2, 3, 4]

In [9]:
import pickle
file = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['pages_code'] # this is a list, len= 53 pages
with open("0001564590-19-038256.txt", "wb") as fp:   # Pickling
    pickle.dump(file, fp)

In [12]:
import pickle 
with open("0001564590-19-038256.txt", "rb") as fp:
    file = pickle.load(fp)
file

['<text>\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n\n<!-- NG Converter v3.0.3.5 -->\n<p style="margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-size:1pt;">\xa0</p>\n<p style="margin-top:0pt;text-align:center;border-top:Solid 1pt;padding-top:1pt;line-height:1pt;border-top:Solid 0.75pt #000000;padding-top:0pt;margin-bottom:0pt;text-indent:0%;font-size:4pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;">\xa0</p>\n<p style="text-align:center;border-top:Solid 1pt;padding-top:1pt;margin-top:1pt;margin-bottom:0pt;text-indent:0%;font-size:1pt;font-family:Times New Roman;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;">\xa0</p>\n<p style="text-align:center;margin-bottom:0pt;margin-top:4pt;text-indent:0%;font-weight:bold;font-size:18pt;font-family:Times New Roman;font-style:normal;text-transform:none;font-variant: normal;">UNITED STATES </p

In [28]:
print(file[3])


<p style="margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-size:1pt;"> </p>
<p style="margin-bottom:0pt;margin-top:0pt;text-indent:0%;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;"><a name="FINANCIAL_INFORMATION"></a>PART I. FINANCIAL INFORMATION </p>
<div align="left">
<table border="0" cellpadding="0" cellspacing="0" style="border-collapse:collapse; width:100%;">
<tr>
<td style="width:10.27%;white-space:nowrap" valign="top">
<p style="margin-bottom:0pt;margin-top:0pt;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;"><a name="Item1_Financial_Statements"></a>I<font style="font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;">TEM 1.</font></p></td>
<td style="width:89.73%;" valign="top">
<p style="margin-bottom:0pt;margin-top:0pt;font-weight:bold;font-style:normal;text-tran

In [38]:
letters = ['abc','bcd','cef','abc', 'abc' ]

abc_len = []

for abc in letters:
    if ('abc' in abc):
        abc_len.append(abc)
        print(abc)

abc
abc
abc


In [39]:
abc_len

['abc', 'abc', 'abc']

In [52]:
# 0001564590-19-038256 I choose this because its the file opened in other tab, @ other desktop. 
page_table_code = []

for page in file:
    
    # if the page has the anchor
    if ('<a name="Consolidated_Balance_Sheets' in page):
        
        print('Consolidated_Balance_Sheets')
        CBS = pd.read_html(page)
        
    if ('<a name="Statements_of_Oper' in page):
        
        print('Statement of Operations')
        SoO = pd.read_html(page)
        
    if ('<a name="Statements_of_Cash' in page):
        
        print('Statement of Cash Flows')
        SoCF = pd.read_html(page)
        # then get all the tables in the page.
        #table = page.find_all('tr')

        #print(table)
        
    else:
        print('something went wrong')
        
    

something went wrong
something went wrong
something went wrong
Consolidated_Balance_Sheets
something went wrong
Statement of Operations
something went wrong
something went wrong
Statement of Cash Flows
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong
something went wrong


In [53]:
CBS

[         0                     1
 0  ITEM 1.  FINANCIAL STATEMENTS,
                                                     0   1            2  \
 0                                                 NaN NaN    March 31,   
 1                                                 NaN NaN         2015   
 2                                                 NaN NaN  (unaudited)   
 3                                              Assets NaN          NaN   
 4                                      Current assets NaN          NaN   
 5                           Cash and cash equivalents NaN            $   
 6           Restricted cash and marketable securities NaN          NaN   
 7                                 Accounts receivable NaN          NaN   
 8                                           Inventory NaN          NaN   
 9           Prepaid expenses and other current assets NaN          NaN   
 10                               Total current assets NaN          NaN   
 11                      Operat

In [ ]:
for acc_num in A_N:

    # first grab all the documents
    filing_documents = master_filings_dict[acc_num]['filing_documents']
    
    # loop through each document
    for document_id in filing_documents:
        
#####################################
# THIS WILL HANDLE THE TABLE SEARCH #
#####################################
        
         # let's grab the normalized text in this example, since it's cleaned and easier to search
        normalized_text_dict = filing_documents[document_id]['pages_normalized_text']  
            
        # define the number of pages
        page_length = len(normalized_text_dict)
        
        # let's grab the all pages code.
        pages_dict = filing_documents[document_id]['pages_code']  
        #with open('{}.csv'.format(acc_num)) as csv_file:

        # initalize a dictionary to store matching words.
        tables_dict = {}

        # loop through each page
        for page_num in pages_dict:

            # grab the actual text
            page_code = pages_dict[int(pages_dict.index(page_num))]
            
            # Convert string to BS object
            page_code = BeautifulSoup(page_code, 'html5')
            
            # search for anchor
            if def table_search(page_code):

                # find all the tables
                tables_found = page_code.find_all('table')

                # save table as dataframe
                df = pd.read_html(str(table))[0]

                # each page is going to be checked, so let's have another dictionary that'll house all the tables found.
                tables_dict[page_num] = {(table_id + 1): table for table_id, table in enumerate(df)}        

                # display a status to the user.
                print(acc_num, ':')
                print('Page {} of {} from document {} contained {} tables.'.format(pages_dict.index(page_num), page_length, document_id, num_found))

        # display a status to the user.  
        print('All the pages from document {} have been scraped for tables.'.format(document_id)) 
        print('-'*80)    

        # let's add the matching tables dict to the document.
        filing_documents[document_id]['table_search'] = tables_dict

In [ ]:
listdf = []
table_rows = table.find_all('tr')

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    listdf.append(row)
pd.DataFrame(l, columns=["A", "B", ...])

# NOT NECESSARY, ATTEMPT WITH ANCHOR USING < a >  tag

table_dictionary = master_filings_dict['0001564590-17-015705']['filing_documents']['10-Q']['table_search']

for document_html in table_dictionary:   
    
    # Convert string to BS object.
    page_code = BeautifulSoup(document_html, 'html5')
    
    anchor = soup.find_all("a", name="CONSOLIDATED_BALANCE_SHEETS")
 
    if page_code == anchor:

        # find table by soup.find_all('table')
        tables_found = page_code.find_all('table')
        print(tables_found)
        break

# NOT NECESSARY, ATTEMPT WITH ANCHOR .find() via using BS4. 

table_dictionary = master_filings_dict['0001564590-17-015705']['filing_documents']['10-Q']['pages_code']

for document_html in table_dictionary:
    result = table_dictionary[table_dictionary.index(document_html)]
    
    # Convert string to BS object
    page_code = BeautifulSoup(result, 'html5')
    
    # define condition to select the table, ie: '<a name="Conso' (<a name=""> is an anchor )
    while page_code.find('a name=') == True:
        
        # find all the tables
        tables_found = page_code.find_all('table')
        print(table_found)
        
        # If the end of article then break.
        break  
        
        # Proceed to the second element in the iterator. 
        continue
        
    # Anything else to execute after the break?
    print('Something went wrong')
    print

# Parse Tables

## Too many table structures in the document.
#### Use IF statement to select Centered Bold text above target tables to 
1) Select, 2) Parse, 3) Store as df in csv file. 

 table_dictionary = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_search']
for num in table_dictionary:
    print(num)
#print(table_dictionary)
#for table_index, table_body in table_dictionary.items():
###    print(table_index, table_body)

table_dictionary = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_search'].values()

# let's grab the all tables code.
#table_dictionary = filing_documents[document_id]['table_search']

### initalize a dictionary to store tables found.
#tables_df = {}

for table_body in table_dictionary:
        
    for num in table_body:

        if 6 < num or num > 4:
            print(num)
            break
        else:
            print(f'this is page_num:', num)
            for table_index, table_values in table_body.items():

                table = table_values.prettify()

                df = pd.read_html(table, header=0)[0]

                print('~'*20)
                print(f'This is the table_index:', table_index)
                display(df)
                print(type(df))
                print('\n')        
                
            break

# >>> Need one more layer of loop

In [ ]:
# Checking what table_search is...
value = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_search'].values()

for item in value:
    print(f'dict_values_keys:', item.keys(), '\n')
    print(f'dict_values_values:', item.values())  
    print('~'*80)

In [ ]:
# Trying code...
tables_dictionary = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_search']

for table_id in tables_dictionary:
            
    # Do the following for each key/table in the dict.
    table_dict_obj = tables_dictionary[table_id]
    
    for table in table_dict_obj:
        
        print(f'this is the key in tables_dictionary["table_search"]:', table)


In [ ]:
# Using the pd.DataFrame() method.
# Test-run

# Retireve the table from the master dictionary.
tables_dictionary = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']

# initalize a dictionary to store tables found.
tables_df = {}

for page_num in tables_dictionary['page_numbers']:
    page_num = int(page_num)
    if 3 < page_num < 8:
        print(f'page_num', page_num)
        
        # Loop through each table in the dictionary.
        for table_id in tables_dictionary['table_search']:
            
            
            # Do the following for each key/table in the dict.
            table_dict_obj = tables_dictionary['table_search'][table_id]
            for table_num in table_dict_obj:

                print(f'this is the key in tables_dictionary["table_search"]', table_num)
            

            soup = BeautifulSoup(str(table_dict_obj), 'html5')
            
            table_rows = soup.find_all('tr')

            # parse the table, first loop through the rows, then each element, and then parse each element.
            parsed_table = [
                [element.get_text(strip=True) for element in row.find_all('td')]
                for row in table_rows
                            ]

            # keep the original just to be safe.
            #new_table_dictionary[table_id]['original_table'] = table_html

            # add the new parsed table.
            #new_table_dictionary[table_id]['parsed_table'] = parsed_table


            # here some additional steps you can take to clean up the data - Removing '$'.
            parsed_table_cleaned = [
                [element for element in row if element != '$']
                for row in parsed_table
                                    ]

            df = pd.DataFrame(parsed_table_cleaned)

            print('~'*80)
            #print(f'This is the page_num:', page_num )
            print(df)
            print('\n')
            del df
            # Creating dictionary object with table_index as keys and df as value.
            # tables_df[table_index] = {(table_idx+1):dfs for table_idx, dfs in enumerate(df)}
            
            #filing_documents[document_id]['table_df'] = tables_df


In [ ]:
# Create dictionary to store parsed_table_cleaned per document id.
for acc_num in A_N:

    # first grab all the documents
    filing_documents = master_filings_dict[acc_num]['filing_documents']
    
    # loop through each document
    for document_id in filing_documents:
        
        # let's grab the all tables code.
        table_dictionary = filing_documents[document_id]['table_search']
        
        # initalize a dictionary to store tables found.
        tables_df = {}
        
        for table_id in table_dictionary:

            table_group = table_dictionary[table_id]

            for table_index, single_table in table_group.items():
                table_rows = single_table.find_all('tr')

                # parse the table, first loop through the rows, then each element, and then parse each element.
                parsed_table = [
                    [element.get_text(strip=True) for element in row.find_all('td')]
                    for row in table_rows
                                ]

                # keep the original just to be safe.
                #new_table_dictionary[table_id]['original_table'] = table_html

                # add the new parsed table.
                #new_table_dictionary[table_id]['parsed_table'] = parsed_table

                # here some additional steps you can take to clean up the data - Removing '$'.
                parsed_table_cleaned = [
                    [element for element in row if element != '$']
                    for row in parsed_table
                                        ]

                df = pd.DataFrame(parsed_table_cleaned)
                tables_df[table_index] = {(table_idx+1):dfs for table_idx, dfs in enumerate(df)}

                filing_documents[document_id]['table_df'] = tables_df
                break

        #print(parsed_table_cleaned)
        #print('~'*40)
        
        
            
    #else:
        
        # if there are no tables then just have the id equal NONE
        #new_table_dictionary[1]['original_table'] = None
        #new_table_dictionary[1]['parsed_table'] = None

In [ ]:
pprint(master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_df'])

In [ ]:
# Test Run.
table_dictionary = master_filings_dict['0001564590-19-038256']['filing_documents']['10-Q']['table_search']

for table_id in table_dictionary:

    table_group = table_dictionary[table_id]
    
    for table_index, single_table in table_group.items():
        table_rows = single_table.find_all('tr')

    # parse the table, first loop through the rows, then each element, and then parse each element.
        parsed_table = [
            [element.get_text(strip=True) for element in row.find_all('td')]
            for row in table_rows
        ]
       
        # keep the original just to be safe.
        #new_table_dictionary[table_id]['original_table'] = table_html
        
        # add the new parsed table.
        #new_table_dictionary[table_id]['parsed_table'] = parsed_table

        # here some additional steps you can take to clean up the data - Removing '$'.
        parsed_table_cleaned = [
            [element for element in row if element != '$']
            for row in parsed_table
        ]
        df = pd.DataFrame(parsed_table_cleaned)
        print(df)

### Not using the function, just looking how to get the tables, I'm modifying the code to suit my needs.

def scrape_table_dictionary(table_dictionary):
    
    # initalize a new dicitonary that'll house all your results
    new_table_dictionary = {}
    
    if len(table_dictionary) != 0:

        # loop through the dictionary
        for table_id in table_dictionary:

            single_table = table_dictionary[table_id]
            # grab the table
            table_html = BeautifulSoup(single_table, 'html5')
            
            # grab all the rows.
            table_rows = table_html.find_all('tr')
            
            # parse the table, first loop through the rows, then each element, and then parse each element.
            parsed_table = [
                [element.get_text(strip=True) for element in row.find_all('td')]
                for row in table_rows
            ]
            
            # keep the original just to be safe.
            new_table_dictionary[table_id]['original_table'] = table_html
            
            # add the new parsed table.
            new_table_dictionary[table_id]['parsed_table'] = parsed_table
            
            # here some additional steps you can take to clean up the data - Removing '$'.
            parsed_table_cleaned = [
                [element for element in row if element != '$']
                for row in parsed_table
            ]
            
            # here some additional steps you can take to clean up the data - Removing Blanks.
            parsed_table_cleaned = [
                [element for element in row if element != None]
                for row in parsed_table_cleaned
            ]

            
    else:
        
        # if there are no tables then just have the id equal NONE
        new_table_dictionary[1]['original_table'] = None
        new_table_dictionary[1]['parsed_table'] = None
        
    return new_table_dictionary

In [ ]:
# Using the scrape_table function method.
for acc_num in A_N:

    # first grab all the documents
    filing_documents = master_filings_dict[acc_num]['filing_documents']
    
    # loop through each document
    for document_id in filing_documents:
        
        # Create new dictionary to store each new_table_dictionary. 
        master_filings_dict[acc_num][document_id]['new_table_dict'] = {}
        
        # dump tables into scrape table function
        scrape_table_dictionary(filing_documents[document_id]['table_search'])
        
        # save the table_dictionary into master dictionary.
        master_filings_dict[acc_num][document_id]['new_table_dict'] = new_table_dictionary

# Clean Dataframe

# Save to local disc as CSV file for import later on

#                                           <<<<< Sigma Code >>>>>

## Defining Our Master Dictionary To House Filings

In [ ]:
# define a dictionary that will house all filings.
master_filings_dict = {}

# let's use the accession number as the key. This 
accession_number = '0001104659-04-027382'

# add a new level to our master_filing_dict, this will also be a dictionary.
master_filings_dict[accession_number] = {}

# this dictionary will contain two keys, the sec header content, and a documents key.
master_filings_dict[accession_number]['sec_header_content'] = {}
master_filings_dict[accession_number]['filing_documents'] = None

## Examing the SEC-Header Tag

In [ ]:
# grab the sec-header tag, so we can store it in the master filing dictionary.
sec_header_tag = soup.find('sec-header')

# store the tag in the dictionary just as is.
master_filings_dict[accession_number]['sec_header_content']['sec_header_code'] = sec_header_tag

# display the sec header tag, so you can see how it looks.
display(sec_header_tag)

## Parsing the documents¶

In [ ]:
# initalize the dictionary that will house all of our documents
master_document_dict = {}

# find all the documents in the filing.
for filing_document in soup.find_all('document'):
    
    # define the document type, found under the <type> tag, this will serve as our key for the dictionary.
    document_id = filing_document.type.find(text=True, recursive=False).strip()
    
    # here are the other parts if you want them.
    document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
    document_filename = filing_document.filename.find(text=True, recursive=False).strip()
    document_description = filing_document.description.find(text=True, recursive=False).strip()
    
    # initalize our document dictionary
    master_document_dict[document_id] = {}
    
    
    # add the different parts, we parsed up above.
    master_document_dict[document_id]['document_sequence'] = document_sequence
    master_document_dict[document_id]['document_filename'] = document_filename
    master_document_dict[document_id]['document_description'] = document_description
    
    # store the document itself, this portion extracts the HTML code. We will have to reparse it later.
    master_document_dict[document_id]['document_code'] = filing_document.extract()
    
    
    # grab the text portion of the document, this will be used to split the document into pages.
    filing_doc_text = filing_document.find('text').extract()

    
    # find all the thematic breaks, these help define page numbers and page breaks.
    all_thematic_breaks = filing_doc_text.find_all('hr',{'width':'100%'})
    
    
    '''
        THE FOLLOWING CODE IS OPTIONAL:
        -------------------------------
        
        This portion will demonstrate how to parse the page number from each "page". Now I would only do this if you
        want the ACTUAL page number on the document, if you don't need it then forget about it and just wait till the
        next seciton.
        
        Additionally, some of the documents appear not to have page numbers when they should so there is no guarantee
        that all the documents will be nice and organized.
    
    '''
    
    
    
    # grab all the page numbers, first one is usually blank
    all_page_numbers = [thematic_break.parent.parent.previous_sibling.previous_sibling.get_text(strip=True) 
                        for thematic_break in all_thematic_breaks]
    
    
    '''
    
        If the above list comprehension doesn't make sense to you, here is how it would look as a regular loop.
    
        # define a list to house all the page numbers
        all_page_numbers = []

        # loop throuhg all the thematic breaks.
        for thematic_break in all thematic_breaks:

           # this would grab the page number tag.
           page_number = thematic_break.parent.parent.previous_sibling.previous_sibling

           # this would grab the page number text
           page_number = page_number.get_text(strip=True)
           
           # store it in the list.
           all_page_numbers.append(page_number)

    '''
    
    # determine the number of pages, will be used for the upcoming if conditions.
    length_of_page_numbers = len(all_page_numbers)
    
    # as long as there are numbers to change then proceed.
    if length_of_page_numbers > 0:
        
        # grab the last number
        previous_number = all_page_numbers[-1]
        
        # initalize a new list
        all_page_numbers_cleaned = []
        
        # loop through the old list in reverse order.
        for number in reversed(all_page_numbers):
            
            # if it's blank proceed to cleaning.
            if number == '':
                
                # the tricky part, there are three scenarios.

                # the previous one we looped was 0 or 1.
                if previous_number == '1' or previous_number == '0':
                    
                    # in this case, it means this is a "new section", so restart at 0.
                    all_page_numbers_cleaned.append(str(0))
                    
                    # reset the page number and the previous number.
                    length_of_page_numbers = length_of_page_numbers - 1
                    previous_number = '0'
                
                # the previous one we looped it wasn't either of those.
                else:
                    
                    # if it was blank, take the current length, subtract 1, and add it to the list.
                    all_page_numbers_cleaned.append(str(length_of_page_numbers - 1))
                    
                    # reset the page number and the previous number.
                    length_of_page_numbers = length_of_page_numbers - 1
                    previous_number = number

            else:
                
                # add the number to the list.
                all_page_numbers_cleaned.append(number)
                
                # reset the page number and the previous number.
                length_of_page_numbers = length_of_page_numbers - 1
                previous_number = number
    else:
        
        # make sure that it has a page number even if there are none, just have it equal 0
        all_page_numbers_cleaned = ['0']
    
    # have the page numbers be the cleaned ones, in reversed order.
    all_page_numbers = list(reversed(all_page_numbers_cleaned))
    
    # store the page_numbers
    master_document_dict[document_id]['page_numbers'] = all_page_numbers
    
    
    '''
        -------------------------------
          THE OPTIONAL CODE HAS ENDED
        -------------------------------
    
        This next portion of code is really what made this all possible. Up above you saw I grabbed all the thematic
        breaks from our document because they sever as natural page breaks. Without those thematic breaks I'm not sure
        if this would be such an easy process. It's not to say we couldn't break it into pages, but I would bet the code
        would be more complex.
    
    '''
    
    
    # convert all thematic breaks to a string so it can be used for parsing
    all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]
    
    # prep the document text for splitting, this means converting it to a string.
    filing_doc_string = str(filing_doc_text)

    
    # handle the case where there are thematic breaks.
    if len(all_thematic_breaks) > 0:
    
        # define the regex delimiter pattern, this would just be all of our thematic breaks.
        regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

        # split the document along each thematic break.
        split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

        # store the document itself
        master_document_dict[document_id]['pages_code'] = split_filing_string

    # handle the case where there are no thematic breaks.
    elif len(all_thematic_breaks) == 0:

        # handles so it will display correctly.
        split_filing_string = all_thematic_breaks
        
        # store the document as is, since there are no thematic breaks. In other words, no splitting.
        master_document_dict[document_id]['pages_code'] = [filing_doc_string]
    

    # display some information to the user.
    print('-'*80)
    print('The document {} was parsed.'.format(document_id))
    print('There was {} page(s) found.'.format(len(all_page_numbers)))
    print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))
    

# store the documents in the master_filing_dictionary.
master_filings_dict[accession_number]['filing_documents'] = master_document_dict

print('-'*80)
print('All the documents for filing {} were parsed and stored.'.format(accession_number))

## Normalizing Text

In [ ]:
# first grab all the documents
filing_documents = master_filings_dict[accession_number]['filing_documents']


# loop through each document
for document_id in filing_documents:
    
    # display some info to give status updates.
    print('-'*80)
    print('Pulling document {} for text normilzation.'.format(document))
    
    # grab all the pages for that document
    document_pages = filing_documents[document_id]['pages_code']
    
    # page length
    pages_length = len(filing_documents[document_id]['pages_code'])
    
    # initalize a dictionary that'll house our repaired html code for each page.
    repaired_pages = {}
    
    # initalize a dictionary that'll house all the normalized text.
    normalized_text = {}

    # loop through each page in that document.
    for index, page in enumerate(document_pages):
        
        # pass it through the parser. NOTE I AM USING THE HTML5 PARSER. YOU MUST USE THIS TO FIX BROKEN TAGS.
        page_soup = BeautifulSoup(page,'html5')
        
        # grab all the text, notice I go to the BODY tag to do this
        page_text = page_soup.html.body.get_text(' ',strip = True)
        
        # normalize the text, remove messy characters. Additionally, restore missing window characters.
        page_text_norm = restore_windows_1252_characters(unicodedata.normalize('NFKD', page_text)) 
        
        # Additional cleaning steps, removing double spaces, and new line breaks.
        page_text_norm = page_text_norm.replace('  ', ' ').replace('\n',' ')
                
        
        '''
            NOTES FROM UP ABOVE:
            --------------------
            
            Remember up above, where I had some optional code. Well, this is where we add page numbers. If you notice
            I simply take the index add 1 to it and with that we now have a page number. Now, this doesn't technically
            follow the sections in each document but I don't think most people will care. Also we will see that we can
            infer the sections from other parts.
        
        '''
        
        # define the page number.
        page_number = index + 1
        
        # add the normalized text to the list.
        normalized_text[page_number] = page_text_norm
        
        # add the repaired html to the list. Also now we have a page number as the key.
        repaired_pages[page_number] = page_soup
    
        # display a status to the user
        print('Page {} of {} from document {} has had their text normalized.'.format(index + 1, 
                                                                                     pages_length, 
                                                                                     document))

    # add the normalized text back to the document dictionary
    filing_documents[document_id]['pages_normalized_text'] = normalized_text
    
    # add the repaired html code back to the document dictionary
    filing_documents[document_id]['pages_code'] = repaired_pages
    
    # define the generated page numbers
    gen_page_numbers = list(repaired_pages.keys())
    
    # add the page numbers we have.
    filing_documents[document_id]['pages_numbers_generated'] = gen_page_numbers    
    
    # display a status to the user.
    print('All the pages from document {} have been normalized.'.format(document_id))


### Additional Scraping and Context Extraction

Alright, so now we are really in a good spot. From here on out, most of what we need to do will be relatively straight forward. By maintaining the natural hierarchy of the filing, and only expanding upon it when beneficial, we can now easily do further analysis of our data.

What I'll demonstrate next is how to get different parts of each page. Additionally, I'll show you strategies for helping to infer the "context" of each page. For example, is a page a signature page, an exhibit page, or a section page? This will help give you context and help you get to where you need to be when getting info.

Also, I'll demonstrate a strategy for how to search for keywords on each page.

Now, like I mentioned up above, this might not be the fastest way to do it, but I am trying to layout this information in a logical format that flows from section to section. I am not concerned about the speed at this point; I'm worried about you following the steps I'm doing to clean my data. I will be posting a more "optimized" script on GitHub in the future.

## Defining Search Words

In [ ]:
search_dict = {
    
    # these could possibly be words that help us find pages that discuss financial statements.
    'financial_words':['liability', 'asset'],
    
    # these could possible be words that help us find sections that discuss administration topics.
    'admin_words':['administration', 'government']
}

In [ ]:
# first grab all the documents
filing_documents = master_filings_dict[accession_number]['filing_documents']

# loop through each document
for document_id in filing_documents:
    
    
    ####################################
    # THIS WILL HANDLE THE WORD SEARCH #
    ####################################
    
    
    # let's grab the normalized text in this example, since it's cleaned and easier to search
    normalized_text_dict = filing_documents[document_id]['pages_normalized_text']  
            
    # initalize a dictionary to store all the tables we find.
    matching_words_dict = {}
    
    # define the number of pages
    page_length = len(normalized_text_dict)
    
    # loop through all the text
    for page_num in normalized_text_dict:
        
        # grab the actual text
        normalized_page_text = normalized_text_dict[page_num]
        
        # each page is going to be checked, so let's have another dictionary that'll house each pages result.
        matching_words_dict[page_num] = {}
        
        # loop through each word list in the search dictionary.
        for search_list in search_dict:
            
            # grab the list of words.
            list_of_words = search_dict[search_list]
            
            # lets see if any of the words are found
            matching_words = [word for word in list_of_words if word in normalized_page_text]
            
            '''
                Again, I know list comprehension might be hard to understand so I'll show you what the loop
                looks like.
                
                # initalize a list of matching words.
                matching_words = []
                
                # loop through the list of words.
                for word in list_of_words:
                
                    # check to see if it's in the text
                    if word in normalized_page_text:
                        
                        # if it is then add it to the list.
                        matching_words.append(word)
            '''
            
            # each page will have a set of results, list of words
            matching_words_dict[page_num][search_list] = {}
            
            # let's add the list of words we search to the matching words dictionary first.
            matching_words_dict[page_num][search_list]['list_of_words'] = list_of_words
            
            # next let's add the list of matchings words to the matching words dictionary.
            matching_words_dict[page_num][search_list]['matches'] = matching_words
            
        
        # display a status to the user.
        print('Page {} of {} from document {} has been searched.'.format(page_num, page_length, document_id))
    
    
    # display a status to the user.
    print('-'*80)    
    print('All the pages from document {} have been searched.'.format(document_id))    
    
    
    ####################################
    # THIS WILL HANDLE THE LINK SEARCH #
    ####################################
    
    
    # let's grab the all pages code.
    pages_dict = filing_documents[document_id]['pages_code']  
            
    # initalize a dictionary to store all the anchors we find.
    link_anchor_dict = {}
    
    # loop through each page
    for page_num in pages_dict:
        
        # grab the actual text
        page_code = pages_dict[page_num]
        
        # find all the anchors in the page, that have the attribute 'name'
        anchors_found = page_code.find_all('a',{'name':True})
        
        # number of anchors found
        num_found = len(anchors_found)
        
        # each page is going to be checked, so let's have another dictionary that'll house all the anchors found.
        link_anchor_dict[page_num]= {(anchor_id + 1): anchor for anchor_id, anchor in enumerate(anchors_found)}        
    
        # display a status to the user.
        print('Page {} of {} from document {} contained {} anchors with names.'.format(page_num, 
                                                                                       page_length, 
                                                                                       document_id, 
                                                                                       num_found))
    
    # display a status to the user.  
    print('All the pages from document {} have been scraped for anchors with names.'.format(document_id)) 
    print('-'*80)  
    
    
    #####################################
    # THIS WILL HANDLE THE TABLE SEARCH #
    #####################################
    
         
    # let's grab the all pages code.
    pages_dict = filing_documents[document_id]['pages_code']  
            
    # initalize a dictionary to store matching words.
    tables_dict = {}
    
    # loop through each page
    for page_num in pages_dict:
        
        # grab the actual text
        page_code = pages_dict[page_num]
        
        # find all the tables
        tables_found = page_code.find_all('table')
        
        # number of tables found
        num_found = len(tables_found)
        
        # each page is going to be checked, so let's have another dictionary that'll house all the tables found.
        tables_dict[page_num] = {(table_id + 1): table for table_id, table in enumerate(tables_found)}        
    
        # display a status to the user.
        print('Page {} of {} from document {} contained {} tables.'.format(page_num, page_length, document_id, num_found))
    
    # display a status to the user.  
    print('All the pages from document {} have been scraped for tables.'.format(document_id)) 
    print('-'*80)    
    
        
    # let's add the matching words dict to the document.
    filing_documents[document_id]['word_search'] = matching_words_dict  
    
    # let's add the matching tables dict to the document.
    filing_documents[document_id]['table_search'] = tables_dict
    
    # let's add the matching anchors dict to the document.
    filing_documents[document_id]['anchor_search'] = link_anchor_dict

## Scraping Tables

In [ ]:
def scrape_table_dictionary(table_dictionary):
    
    # initalize a new dicitonary that'll house all your results
    new_table_dictionary = {}
    
    if len(table_dictionary) != 0:

        # loop through the dictionary
        for table_id in table_dictionary:

            # grab the table
            table_html = table_dictionary[table_id]
            
            # grab all the rows.
            table_rows = table_html.find_all('tr')
            
            # parse the table, first loop through the rows, then each element, and then parse each element.
            parsed_table = [
                [element.get_text(strip=True) for element in row.find_all('td')]
                for row in table_rows
            ]
            
            # keep the original just to be safe.
            new_table_dictionary[table_id]['original_table'] = table_html
            
            # add the new parsed table.
            new_table_dictionary[table_id]['parsed_table'] = parsed_table
            
            # here some additional steps you can take to clean up the data - Removing '$'.
            parsed_table_cleaned = [
                [element for element in row if element != '$']
                for row in parsed_table
            ]
            
            # here some additional steps you can take to clean up the data - Removing Blanks.
            parsed_table_cleaned = [
                [element for element in row if element != None]
                for row in parsed_table_cleaned.
            ]

            
    else:
        
        # if there are no tables then just have the id equal NONE
        new_table_dictionary[1]['original_table'] = None
        new_table_dictionary[1]['parsed_table'] = None
        
    return new_table_dictionary

Complex Searches
In some cases, what you're trying to do might require doing a very complex search. For example, we might want the text that is centered over the page. This will give us context as to what is on the page and provide us more details of its content. This would fall under a "complex search," and to do a complex search, we will need to define some extra strategies we can take to find different parts of the page.

The beautiful thing about BeautifulSoup is we can define functions where we set our criteria and then only return tags that match those criteria. Below I've provided some example "search functions" that will help you find precise portions of your page.

Search For Centered Headers
In [ ]:
def search_for_centered_headers(tag):

    # easy way to end early is check if the 'align' keet is in attributes.
    if 'align' not in tag.attrs:
        return
    
    # define the criteria.
    criteria1 = tag.name == 'p'                # I want the tag to be name of 'p'
    criteria2 = tag.parent.name != 'td'        # I want the parent tag NOT to be named 'td'
    criteria3 = tag['align'] == 'center'       # I want the 'align' attribute to be labeled 'center'.
    
    # if it matches all the criteria then return the text.
    if criteria1 and criteria2 and criteria3:         
        return tag.get_text(strip = True)
Search For Bolded Text
In [ ]:
def search_for_bolded_tags(tag):
    
    # define the criteria.
    criteria1 = tag.name == 'b'                # I want the tag to be name of 'p'
    criteria2 = tag.parent.name != 'td'        # I want the parent tag NOT to be named 'td'
    
    # if it matches all the criteria then return the text.
    if criteria1 and criteria2:         
        return tag.get_text(strip = True).replace('\n',' ')
Using Custom Search Functions
In [ ]:
# first grab all the documents
filing_documents = master_filings_dict[accession_number]['filing_documents']

# loop through each document
for document_id in filing_documents:   
    
    # let's grab the all pages code.
    pages_dict = filing_documents[document_id]['pages_code']  
            
    # initalize a dictionary to store all the anchors we find.
    centered_headers_dict = {}
    
    # loop through each page
    for page_num in pages_dict:
        
        # grab the actual text
        page_code = pages_dict[page_num]
        
        # find all the anchors in the page, that have the attribute 'name'
        centered_headers_found = page_code.find_all(search_for_centered_headers)
        
        # number of anchors found
        num_found = len(centered_headers_found)
   
        # display a status to the user.
        print('Page {} of {} from document {} contained {} centered headers.'.format(page_num, 
                                                                                     page_length, 
                                                                                     document_id, 
                                                                                     num_found))
    
    # display a status to the user.  
    print('All the pages from document {} have been scraped for centered headers.'.format(document_id)) 
    print('-'*80)


In [ ]:
list_a = ['a','b','c','d','e']

for letter in list_a:
    result = list_a[list_a.index(letter)]
    print(result, ':)' )

In [ ]:
list_a[0]
